Loading Data:

In [361]:
import pandas as pd

# Load the Excel file into a DataFrame
data = pd.read_excel("Survey_ICS487_Project.xlsx")


Data Inspection:

In [362]:
# Check the first few rows
print(data.head())

# Get an overview of the data
print(data.info())

# Check for missing values
print(data.isnull().sum())


         Name  Salary (SAR)  Age    State     Sex  Monthly Debt (SAR)  \
0     Ibrahim          3341   30   Single  Female              824.26   
1  abdulmajid          2595   27  Married    Male              362.05   
2      Bander          2472   26  Married    Male              204.87   
3        sami          2841   57  Married    Male              464.94   
4       salem          2337   54   Single    Male              507.02   

   Elementary Expenses (SAR)        Goal  Number of Children  \
0                    2607.09  Investment                   0   
1                    4919.38     Savings                   5   
2                    4152.50     Savings                   3   
3                    5733.83     Savings                   0   
4                    4012.42  Investment                   0   

  Employment_Status Nationality Budgeting_Rule Financial_Comfortability  \
0           Student       Saudi       50/30/20                   ['no']   
1           Student       

Clean and Preprocess:

In [363]:
# Standardize column names
data.columns = [col.strip().lower().replace(" ", "_") for col in data.columns]
# Check for duplicates and remove them
data = data.drop_duplicates()

# Handle missing values (drop rows or fill them as needed)
data = data.dropna()  # Drop rows with missing values
# Alternatively, fill missing values (example: fill salary with median)
data['salary_(sar)'] = data['salary_(sar)'].fillna(data['salary_(sar)'].median())

# Ensure numeric columns are properly formatted
numeric_columns = ['salary_(sar)', 'age', 'monthly_debt_(sar)', 'elementary_expenses_(sar)']
for col in numeric_columns:
    data[col] = pd.to_numeric(data[col], errors='coerce')

# Validate ranges for numeric columns
data = data[(data['age'] >= 18) & (data['age'] <= 100)]  # Age should be between 0 and 100
data = data[data['salary_(sar)'] >= 0]  # Salary should not be negative

# Filter out rows where the conditions are met
filtered_data = data[
    (data['nationality'].str.lower() == 'saudi') ]  # Keep only rows where nationality is 'saudi'
filtered_data = filtered_data[
    (filtered_data['financial_comfortability'].str.lower() != 'no') ]  # Exclude rows where financial_comfortability is 'no'
filtered_data = filtered_data[
    (data['goal_progress'].str.lower() != 'no')  # Exclude rows where goal_progress is 'no'
]

# Drop the irrelevant columns
filtered_data = filtered_data.drop(columns=['nationality', 'financial_comfortability', 'goal_progress'])

# Store the filtered data back if needed
data = filtered_data
print(data.tail())



       name  salary_(sar)  age    state     sex  monthly_debt_(sar)  \
1171   Yara         21726   36  Married  Female             1275.94   
1172  Malik           951   23   Single    Male              271.78   
1173   Saif         32399   49  Married    Male             2906.62   
1174  Malik          2966   49   Single  Female              304.50   
1175  fares          2622   48  Married    Male              165.59   

      elementary_expenses_(sar)        goal  number_of_children  \
1171                    7075.70  Investment                   0   
1172                    1753.42  Investment                   0   
1173                    7802.89  Investment                   2   
1174                    2421.45     Savings                   0   
1175                    9368.64     Savings                   5   

     employment_status budgeting_rule  
1171          Employed       50/30/20  
1172           Student       50/30/20  
1173          Employed       60/20/20  
1174      

/var/folders/yc/7vmb_7z55z3d92njbdlxxr3m0000gp/T/ipykernel_61078/1195411881.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_data = filtered_data[


Removing outliers:

In [364]:
import numpy as np

def remove_outliers_iqr(df, column):
    """
    Removes outliers from a column using the IQR method.
    """
    Q1 = df[column].quantile(0.25)  # First quartile
    Q3 = df[column].quantile(0.75)  # Third quartile
    IQR = Q3 - Q1  # Interquartile range
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

# Apply to numeric columns with potential outliers
numeric_columns = ['salary_(sar)', 'age', 'monthly_debt_(sar)', 'elementary_expenses_(sar)']
for col in numeric_columns:
    data = remove_outliers_iqr(data, col)


# Define age bins and labels
age_bins = [0, 24, 34, 44, 54, float('inf')]
age_labels = ['18-24', '25-34', '35-44', '45-54', '55+']

# Create a new column for age groups
data['age_group'] = pd.cut(data['age'], bins=age_bins, labels=age_labels, right=True)

data = data.drop(columns=['age'])


Encoding Categorical Variables:

In [365]:
from sklearn.preprocessing import LabelEncoder

# One-hot encode categorical columns
categorical_columns = ['state', 'sex', 'goal', 'employment_status']
data = pd.get_dummies(data, columns=categorical_columns, drop_first=True)

# Encode the age_group using LabelEncoder
label_encoder = LabelEncoder()
data['age_group_encoded'] = label_encoder.fit_transform(data['age_group'])

# Display the encoding
print("Age Group Encoding:")
for label, encoded in zip(label_encoder.classes_, range(len(label_encoder.classes_))):
    print(f"{label}: {encoded}")

data = data.drop(columns=['age_group'])


Age Group Encoding:
18-24: 0
25-34: 1
35-44: 2
45-54: 3
55+: 4


Feature Engineering:

In [366]:
# Create new features
data['debt_to_income_ratio'] = data['monthly_debt_(sar)'] / data['salary_(sar)']
data['savings_ratio'] = data['elementary_expenses_(sar)'] / data['salary_(sar)']

# Handle infinite or NaN values due to division
data['debt_to_income_ratio'] = data['debt_to_income_ratio'].replace([np.inf, -np.inf], np.nan).fillna(0)
data['savings_ratio'] = data['savings_ratio'].replace([np.inf, -np.inf], np.nan).fillna(0)

# Drop irrelevant columns
data = data.drop(columns=['name'])

data.describe()

,salary_(sar),monthly_debt_(sar),elementary_expenses_(sar),number_of_children,age_group_encoded,debt_to_income_ratio,savings_ratio
count,717.000000,717.000000,717.000000,717.000000,717.000000,717.000000,717.000000
mean,3776.023710,490.994826,5699.952720,1.234310,1.343096,0.153535,2.376135
std,3742.614289,387.375079,3650.813219,1.706313,1.256046,0.080978,2.135130
min,910.000000,50.990000,467.220000,0.000000,0.000000,0.007880,0.027823
25%,1817.000000,209.130000,2695.560000,0.000000,0.000000,0.082876,0.766568
50%,2541.000000,388.330000,4831.890000,0.000000,1.000000,0.151888,1.752422
75%,3808.000000,651.050000,8662.680000,2.000000,2.000000,0.221979,3.272212
max,22117.000000,1853.370000,14564.850000,5.000000,4.000000,0.299759,11.825868


Saving Data:

In [367]:
# Save cleaned data to a new Excel file
data.to_excel("Cleaned_Survey_ICS487_Project.xlsx", index=False)

# Or save it as a CSV file
data.to_csv("Cleaned_Survey_ICS487_Project.csv", index=False)